In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Pheochromocytoma_and_Paraganglioma"
cohort = "GSE67066"

# Input paths
in_trait_dir = "../../input/GEO/Pheochromocytoma_and_Paraganglioma"
in_cohort_dir = "../../input/GEO/Pheochromocytoma_and_Paraganglioma/GSE67066"

# Output paths
out_data_file = "../../output/preprocess/Pheochromocytoma_and_Paraganglioma/GSE67066.csv"
out_gene_data_file = "../../output/preprocess/Pheochromocytoma_and_Paraganglioma/gene_data/GSE67066.csv"
out_clinical_data_file = "../../output/preprocess/Pheochromocytoma_and_Paraganglioma/clinical_data/GSE67066.csv"
json_path = "../../output/preprocess/Pheochromocytoma_and_Paraganglioma/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Assess gene expression data availability
is_gene_available = True  # From the background information, we can see that mRNA expression arrays were performed

# 2. Assess variable availability and create conversion functions

# 2.1 Trait (Malignancy)
trait_row = 1  # "tissue type: benign" or "tissue type: malignant"

def convert_trait(value):
    if not isinstance(value, str):
        return None
    value = value.lower()
    if 'malignant' in value:
        return 1  # Malignant
    elif 'benign' in value:
        return 0  # Benign
    return None

# 2.2 Age
age_row = None  # Age information is not available in the sample characteristics

def convert_age(value):
    # Placeholder function since age data is not available
    return None

# 2.3 Gender
gender_row = None  # Gender information is not available in the sample characteristics

def convert_gender(value):
    # Placeholder function since gender data is not available
    return None

# 3. Save metadata (initial filtering)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if trait data is available
if trait_row is not None:
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the clinical dataframe
    preview = preview_df(clinical_df)
    print("Clinical data preview:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
# Reviewing the gene identifiers
# These look like Affymetrix probe IDs (for example "1007_s_at", "1053_at") rather than standard human gene symbols
# Human gene symbols would typically be like BRCA1, TP53, etc.
# Affymetrix IDs need to be mapped to gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# Looking at the gene annotation data, we can see that:
# - "ID" contains Affymetrix probe IDs like '1007_s_at', '1053_at', etc. (same format as in gene_data)
# - "Gene Symbol" contains human gene symbols like 'DDR1 /// MIR4640', 'RFC2', etc.

# 1. Identify the columns in the annotation data for mapping
probe_col = 'ID'  # Column with probe identifiers
gene_col = 'Gene Symbol'  # Column with gene symbols

# 2. Create the gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, probe_col, gene_col)

# Preview the mapping dataframe
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply the gene mapping to convert probe-level data to gene-level data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the gene expression data after mapping
print("\nGene expression data after mapping (first 5 genes, 5 samples):")
preview_gene_data = gene_data.iloc[:5, :5]
print(preview_gene_data)


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data from step 2
clinical_data = pd.read_csv(out_clinical_data_file)
# Convert to transposed format for linking (samples as columns)
clinical_df = pd.DataFrame(clinical_data.values.T, columns=[trait], index=clinical_data.columns)
print(f"Loaded clinical data with shape: {clinical_df.shape}")

# Link the clinical and genetic data on sample IDs
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=f"Dataset contains gene expression data for pheochromocytoma/paraganglioma samples with benign/malignant classification."
)

# 6. Save the data if it's usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the actual clinical data that was created in step 2
clinical_data = pd.read_csv(out_clinical_data_file)
# Since the clinical data's columns are sample IDs and the only row is the trait data
# We need to transpose it to have samples as rows and traits as columns
clinical_df = pd.DataFrame(clinical_data.T.values, 
                           index=clinical_data.columns,
                           columns=[trait])
print(f"Loaded clinical data with shape: {clinical_df.shape}")

# Link the clinical and genetic data on sample IDs
linked_data = pd.concat([clinical_df, normalized_gene_data.T], axis=1)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=f"Dataset contains gene expression data for pheochromocytoma/paraganglioma samples with benign/malignant classification."
)

# 6. Save the data if it's usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")